In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_files
#from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K

2024-02-09 22:06:48.242770: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 22:06:48.242893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 22:06:48.415812: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_data='/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training'

In [3]:
test_data='/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test'

In [4]:
def get_data(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files,targets,target_labels

X_train, Y_train, labels = get_data(train_data)
X_test, Y_test,_ = get_data(test_data)

In [5]:
from keras.utils import to_categorical

In [6]:
Y_train = to_categorical(Y_train, 131)
Y_test = to_categorical(Y_test, 131)

In [7]:
X_train, X_val = train_test_split(X_train, test_size=0.2, random_state=33)
Y_train, Y_val = train_test_split(Y_train, test_size=0.2, random_state=33)

In [8]:
def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

In [9]:
X_train = np.array(convert_image_to_array(X_train))
X_val = np.array(convert_image_to_array(X_val))
X_test = np.array(convert_image_to_array(X_test))

In [10]:
X_train = X_train.astype('float32')/255
X_val = X_val.astype('float32')/255
X_test = X_test.astype('float32')/255

In [11]:
# Mobilenet Backbone
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense

# Model Training
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

IMAGE_SIZE = 100
print("Loading MobileNet Backbone: ")#APPLYING MOBILENETV2 MODEL 
mobilenet = MobileNetV2(input_shape=(100,100, 3), weights='imagenet', include_top=False)

# Freeze the model weights
mobilenet.trainable = False

# The Mobilenet Model baseline
#current model contains moile
mobilenet = Sequential([
    mobilenet,
    GlobalAveragePooling2D(),
    Dropout(0.4),
    Dense(131, activation='softmax')
])

# Compile the Baseline
mobilenet.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# Train the Xception Baseline Model
print("\nTraining Baseline Model: ")
mobilenet.fit(
    X_train, Y_train, 
    validation_data=(X_val, Y_val), 
    epochs=20, 
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True),
        ModelCheckpoint("MobileNetV2Baseline.h5", save_best_only=True)
    ],
    batch_size=128
)

# Testing Evaluation
mobilenet_test_loss, mobilenet_test_acc = mobilenet.evaluate(X_test, Y_test)
print(f"\nMobileNet Baseline Testing Loss     : {mobilenet_test_loss}.")
print(f"MobileNet Baseline Testing Accuracy : {mobilenet_test_acc}.")

Loading MobileNet Backbone: 

Training Baseline Model: 
Epoch 1/20
  1/424 [..............................] - ETA: 49:18 - loss: 7.1282 - accuracy: 0.0078

I0000 00:00:1707516890.517765     364 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


424/424 [==============================] - ETA: 0s - loss: 0.5796 - accuracy: 0.8625

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


424/424 [==============================] - 34s 63ms/step - loss: 0.5796 - accuracy: 0.8625 - val_loss: 0.0307 - val_accuracy: 0.9970
Epoch 2/20
424/424 [==============================] - 20s 48ms/step - loss: 0.0534 - accuracy: 0.9892 - val_loss: 0.0114 - val_accuracy: 0.9993
Epoch 3/20
424/424 [==============================] - 21s 49ms/step - loss: 0.0265 - accuracy: 0.9951 - val_loss: 0.0060 - val_accuracy: 0.9997
Epoch 4/20
424/424 [==============================] - 21s 49ms/step - loss: 0.0159 - accuracy: 0.9976 - val_loss: 0.0038 - val_accuracy: 0.9998
Epoch 5/20
424/424 [==============================] - 21s 49ms/step - loss: 0.0115 - accuracy: 0.9982 - val_loss: 0.0026 - val_accuracy: 0.9999
Epoch 6/20
424/424 [==============================] - 21s 49ms/step - loss: 0.0085 - accuracy: 0.9989 - val_loss: 0.0020 - val_accuracy: 0.9999
Epoch 7/20
424/424 [==============================] - 21s 49ms/step - loss: 0.0072 - accuracy: 0.9990 - val_loss: 0.0013 - val_accuracy: 0.9999
Epo